In [1]:
from itertools import permutations
import numpy as np
import math

In [2]:
# Generate all possible 3x3 grids using numbers 1 to 9:
def generate_grids():
    return [np.array(p).reshape(3, 3) for p in permutations(range(1, 10))]

In [3]:
#Define the relevant D4 transformations:
def rotate_grid(grid):
    # Rotate the grid by 90 degrees
    return np.rot90(grid)

def reflect_grid(grid, axis):
    # Reflect the grid over a specified axis ('vertical' or 'horizontal')
    if axis == 'vertical':
        return np.fliplr(grid)
    elif axis == 'horizontal':
        return np.flipud(grid)

In [4]:
# Apply all D4 transformations to the grid and return the set of unique grids
def apply_transformations(grid):
    # Apply all D4 transformations to the grid and return the set of unique grids
    transformations = {tuple(grid.flatten())}
    for _ in range(3):
        grid = rotate_grid(grid)
        transformations.add(tuple(grid.flatten()))
    transformations.add(tuple(reflect_grid(grid, 'vertical').flatten()))
    transformations.add(tuple(reflect_grid(grid, 'horizontal').flatten()))
    grid = reflect_grid(grid, 'vertical')  # Reflect back for diagonal reflections
    transformations.add(tuple(np.fliplr(np.rot90(grid)).flatten()))
    transformations.add(tuple(np.rot90(np.fliplr(grid)).flatten()))
    return transformations



In [5]:
# Function to find unique grids that generate their 'class'
def find_unique_grids(grids):
    unique_grids = set()
    seen_transformations = set()

    for grid in grids:
        transformations = apply_transformations(grid)
        if not seen_transformations.intersection(transformations):
            unique_grids.add(tuple(grid.flatten()))
            seen_transformations.update(transformations)

    return [np.array(grid).reshape(3, 3) for grid in unique_grids]


In [6]:
grids = generate_grids()
unique_grids = find_unique_grids(grids)
print(f"Number of unique grids after applying D4 transformations: {len(unique_grids)}")

Number of unique grids after applying D4 transformations: 45360


In [7]:
#Funtion that checks which of these grids have a possible 1-9 path
def is_adjacent(grid, num1, num2):
    pos1 = np.argwhere(grid == num1)
    pos2 = np.argwhere(grid == num2)

    if not pos1.size or not pos2.size:  # Check if either position is missing
        return False

    pos1, pos2 = pos1[0], pos2[0]
    return np.max(np.abs(pos1 - pos2)) <= 1

def find_adjacent_pairs(grid):
    adjacent_pairs = []
    for num in range(1, 9):  # Check pairs 1-2 to 8-9
        if is_adjacent(grid, num, num + 1):
            adjacent_pairs.append((num, num + 1))
    return adjacent_pairs

def find_largest_path_length(adjacent_pairs):
    if not adjacent_pairs:
        return 0

    # Sort pairs to ensure they are in ascending order
    adjacent_pairs.sort()
    max_length = 1
    current_length = 1

    for i in range(1, len(adjacent_pairs)):
        if adjacent_pairs[i][0] == adjacent_pairs[i-1][1]:  # Check if pairs are connected
            current_length += 1
            max_length = max(max_length, current_length)
        else:
            current_length = 1  # Reset if pairs are not connected

    return max_length


In [8]:
path_length_counters = {i: 0 for i in range(1, 10)}

In [9]:
for grid in unique_grids:
    adjacent_pairs = find_adjacent_pairs(grid)
    max_path_length = find_largest_path_length(adjacent_pairs)
    path_length_counters[max_path_length] += 1

print("Path length counters:", path_length_counters)


Path length counters: {1: 2724, 2: 17170, 3: 14066, 4: 6918, 5: 2840, 6: 1144, 7: 400, 8: 98, 9: 0}


In [10]:
#Finding the average length
def calculate_expected_value(path_counters,unique_grids):
    expected_value = 0

    for path_length, counter in path_counters.items():
        # Calculate the probability for this path length
        cal = (path_length+1)*counter*8*(1/2)**(path_length)+1
        # Add to the expected value the product of path length and its probability
        expected_value += cal

    return expected_value/len(unique_grids)


In [11]:
expected_value = calculate_expected_value(path_length_counters, unique_grids)
print("Expected value of path length:", expected_value)

Expected value of path length: 4.494456845238095
